In [22]:
import glob                         # this module helps in selecting files 
import pandas as pd                 # this module helps in processing CSV files
import xml.etree.ElementTree as ET  # this module helps in processing XML files.
from datetime import datetime

## Download Files

In [23]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip

--2023-04-26 18:55:36--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4249 (4.1K) [application/zip]
Saving to: ‘datasource.zip.1’

datasource.zip.1    100%[===================>]   4.15K  --.-KB/s    in 0s      

2023-04-26 18:55:39 (298 MB/s) - ‘datasource.zip.1’ saved [4249/4249]



In [24]:
!unzip datasource.zip -d dealership_data

Archive:  datasource.zip
replace dealership_data/used_car_prices1.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
logfile = "dealership_logfile.txt"
targetfile="dealership_transformed_data.csv" 

In [ ]:
#extract 
def extract_from_csv(file_to_process):
    df=pd.read_csv(file_to_process)
    return df

def extract_from_json(file_to_process):
    df=pd.read_json(file_to_process, lines=True)
    return df

def extract_from_xml(file_to_process):
    df=pd.DataFrame(columns=["car_model","year_of_manufacture","price","fuel"])
    tree=ET.parse(file_to_process)
    root=tree.getroot()
    for car in root:
        car_model=car.find("car_model").text
        year_of_manufacture=int(car.find("year_of_manufacture").text)
        price=float(car.find("price").text)
        fuel=car.find("fuel").text
        df=df.append({"car_model":car_model,"year_of_manufacture":year_of_manufacture,"price":price,"fuel":fuel},ignore_index=True)
    return df 


In [ ]:
#extract function
def extract():
    extracted_data=pd.DataFrame(columns=["car_model","year_of_manufacture","price","fuel"])

    for csvfile in glob.glob('dealership_data/*.csv'):
        extracted_data=extracted_data.append(extract_from_csv(csvfile), ignore_index=True)

    for jsonfile in glob.glob('dealership_data/*.json'):
        extracted_data=extracted_data.append(extract_from_json(jsonfile), ignore_index=True)

    for xmlfile in glob.glob('dealership_data/*.xml'):
        extracted_data=extracted_data.append(extract_from_xml(xmlfile), ignore_index=True)

    return extracted_data

In [ ]:
#transform data
def transform(data):
        data['price'] = round(data.price, 2)
        return data

In [ ]:
def load(targetfile,file_to_load):
    file_to_load.to_csv(targetfile)
    


In [ ]:
#logging
def log(message):
    timestamp_format='%Y-%h-%d-%H:%M:%S'
    now=datetime.now()
    timestamp=now.strftime(timestamp_format)
    with open("logfile.txt","a") as f:
        f.write(timestamp +' , '+ message + '\n')

## Running ETL process

In [ ]:
log("ETL Job Started")

In [ ]:
log("Extract phase started")
extracted_data=extract()
log("extract phase ended")
extracted_data

/tmp/ipykernel_8816/1835644807.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data=extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
/tmp/ipykernel_8816/1835644807.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data=extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
/tmp/ipykernel_8816/1835644807.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data=extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
/tmp/ipykernel_8816/1835644807.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data=extracted_data.append(extract_from_json(jsonfile

,car_model,year_of_manufacture,price,fuel
0,alto 800,2017,4253.731343,Petrol
1,ciaz,2015,10223.880597,Diesel
2,ciaz,2015,11194.029851,Petrol
3,ertiga,2015,9104.477612,Petrol
4,dzire,2009,3358.208955,Petrol
...,...,...,...,...
85,etios liva,2014,7089.552239,Diesel
86,innova,2017,29477.611940,Petrol
87,fortuner,2010,13805.970149,Diesel
88,corolla altis,2011,6492.537313,Petrol


In [ ]:
log("transform phase started")
transformed_data=transform(extracted_data)
log("tranform phase complete")
transformed_data

,car_model,year_of_manufacture,price,fuel
0,alto 800,2017,4253.73,Petrol
1,ciaz,2015,10223.88,Diesel
2,ciaz,2015,11194.03,Petrol
3,ertiga,2015,9104.48,Petrol
4,dzire,2009,3358.21,Petrol
...,...,...,...,...
85,etios liva,2014,7089.55,Diesel
86,innova,2017,29477.61,Petrol
87,fortuner,2010,13805.97,Diesel
88,corolla altis,2011,6492.54,Petrol


In [ ]:
log("load phase started")
load(targetfile,transformed_data)
log("load phase ended")



In [ ]:
log("etl job ended")